<a href="https://colab.research.google.com/github/Waynel65/deep-learning-project/blob/main/CS200_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**List of food that can be recongized**

*   apple pie
*   beignets
*   bruschetta
*   chicken wings
*   chocolate cakes
*   egg benedict
*   fried calamari
*   fried rice
*   greek salad
*   guacamole
*   hamburger
*   miso soup
*   pizza
*   pork chop
*   prime rib
*   ramen
*   risotto
*   sashimi


In [ ]:
## mounting google drive on google colab
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
## check if the drive is mounted correctly
PATH_OF_DATA = '/content/drive/"My Drive"/foodImages'
!ls {PATH_OF_DATA}


In [ ]:
## all the imports
import torch
import torch.optim as optim
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import torchvision.models as models
import torchvision.transforms as transforms
import torchvision.datasets as datasets
import matplotlib.pyplot as plt
from torchvision.datasets import ImageFolder
from tqdm import tqdm

In [ ]:
## hyperparameters
BATCH_SIZE = 100
LEARNING_RATE = 0.005
NUM_EPOCHs = 10

In [ ]:
## data loading
## challenge 1: could not divide the image data into training and testing set
## challenge 2: each category has different number of samples

img_transform = transforms.Compose([transforms.Resize((384, 384)), 
                               transforms.ToTensor(),
                               transforms.Normalize([0.485, 0.456, 0.406] , [0.229, 0.224, 0.225])]
)

img_dataset = datasets.ImageFolder(root = "/content/drive/My Drive/foodImages",
                                   transform=img_transform)
img_dataloader = torch.utils.data.DataLoader(img_dataset, batch_size = BATCH_SIZE,
                                              shuffle = True, num_workers = 2)

In [ ]:
## pretrained model
googlenet = models.googlenet(pretrained=True)

In [ ]:
## neural network class
class Network(nn.Module):
  def __init__(self):
    super().__init__()
    self.googlenet = googlenet

    for p in self.googlenet.parameters():
      p.require_grad = False  
    
    self.googlenet.fc = nn.Linear(1024, 18)
    self.googlenet.fc.requires_grad = True


  def forward(self, x):
    return self.googlenet(x)


In [ ]:
network = Network().cuda()  ## network instance
loss_fn = nn.CrossEntropyLoss()
optimizer = optim.Adam(network.parameters(), lr=LEARNING_RATE)

def num_correct(preds, labels):
  return preds.argmax(dim=1).eq(labels).sum().item()

In [ ]:
for epoch in range(NUM_EPOCHs):
  total_correct = 0.0
  total_loss = 0.0

  for data in tqdm(img_dataloader, total=len(img_dataloader)):
    images, labels = data
    images, labels = images.cuda(), labels.cuda()

    optimizer.zero_grad()

    preds = network(images) ## prediction tensors
    loss = loss_fn (preds, labels) ## calculate loss
    loss.backward()
    optimizer.step() ## update the weights (learning rate tells how far to step in the direction of local minimum)

    total_loss += loss.item()
    total_correct += num_correct(preds, labels)

  print("Epoch: ", epoch, "total correct: ", total_correct, "total loss: ", total_loss)

In [ ]:
print(total_correct/len(img_dataset))
